In [ ]:
!pip install pandas numpy scipy scikit-learn tensorflow transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import torch
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import normalize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/myntra dataset/Myntra Fashion Clothing.csv')

In [ ]:
df

,URL,Product_id,BrandName,Category,Individual_category,category_by_Gender,Description,DiscountPrice (in Rs),OriginalPrice (in Rs),DiscountOffer,SizeOption,Ratings,Reviews
0,https://www.myntra.com/jeans/roadster/roadster...,2296012,Roadster,Bottom Wear,jeans,Men,roadster men navy blue slim fit mid rise clean...,824.0,1499,45% OFF,"28, 30, 32, 34, 36",3.9,999
1,https://www.myntra.com/track-pants/locomotive/...,13780156,LOCOMOTIVE,Bottom Wear,track-pants,Men,locomotive men black white solid slim fit tra...,517.0,1149,55% OFF,"S, M, L, XL",4.0,999
2,https://www.myntra.com/shirts/roadster/roadste...,11895958,Roadster,Topwear,shirts,Men,roadster men navy white black geometric print...,629.0,1399,55% OFF,"38, 40, 42, 44, 46, 48",4.3,999
3,https://www.myntra.com/shapewear/zivame/zivame...,4335679,Zivame,Lingerie & Sleep Wear,shapewear,Women,zivame women black saree shapewear zi3023core0...,893.0,1295,31% OFF,"S, M, L, XL, XXL",4.2,999
4,https://www.myntra.com/tshirts/roadster/roadst...,11690882,Roadster,Western,tshirts,Women,roadster women white solid v neck pure cotton ...,NaN,599,35% OFF,"XS, S, M, L, XL",4.2,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,https://www.myntra.com/jeans/roadster/roadster...,11274476,Roadster,Bottom Wear,jeans,Men,roadster men blue skinny fit mid rise acid was...,NaN,1999,NaN,"28, 30, 32, 34, 36",3.8,904
495,https://www.myntra.com/shirts/roadster/roadste...,1376192,Roadster,Topwear,shirts,Men,roadster men blue pure cotton washed denim sus...,719.0,1799,60% OFF,"38, 40, 42, 44",4.1,904
496,https://www.myntra.com/dresses/tokyo-talkies/t...,10088851,Tokyo Talkies,Western,dresses,Women,tokyo talkies women rust red floral printed dress,NaN,1399,66% OFF,"S, M, L, XL",4.2,904
497,https://www.myntra.com/tops/her-by-invictus/he...,10821238,her by invictus,Western,tops,Women,her by invictus teal blue black conversationa...,NaN,1399,65% OFF,"S, M, L, XL, XXL",3.8,904


In [ ]:
print("Dataset Preview:")
print(df.head())

Dataset Preview:
                                                 URL  Product_id   BrandName  \
0  https://www.myntra.com/jeans/roadster/roadster...     2296012    Roadster   
1  https://www.myntra.com/track-pants/locomotive/...    13780156  LOCOMOTIVE   
2  https://www.myntra.com/shirts/roadster/roadste...    11895958    Roadster   
3  https://www.myntra.com/shapewear/zivame/zivame...     4335679      Zivame   
4  https://www.myntra.com/tshirts/roadster/roadst...    11690882    Roadster   

                Category Individual_category category_by_Gender  \
0            Bottom Wear               jeans                Men   
1            Bottom Wear         track-pants                Men   
2                Topwear              shirts                Men   
3  Lingerie & Sleep Wear           shapewear              Women   
4                Western             tshirts              Women   

                                         Description  DiscountPrice (in Rs)  \
0  roadster men navy

In [ ]:
df.shape

(499, 13)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   URL                    499 non-null    object 
 1   Product_id             499 non-null    int64  
 2   BrandName              499 non-null    object 
 3   Category               499 non-null    object 
 4   Individual_category    499 non-null    object 
 5   category_by_Gender     499 non-null    object 
 6   Description            499 non-null    object 
 7   DiscountPrice (in Rs)  266 non-null    float64
 8   OriginalPrice (in Rs)  499 non-null    int64  
 9   DiscountOffer          446 non-null    object 
 10  SizeOption             499 non-null    object 
 11  Ratings                499 non-null    float64
 12  Reviews                499 non-null    int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 50.8+ KB


In [ ]:
df.describe()

,Product_id,DiscountPrice (in Rs),OriginalPrice (in Rs),Ratings,Reviews
count,4.990000e+02,266.000000,499.000000,499.000000,499.000000
mean,9.578347e+06,774.699248,1623.847695,4.139279,950.176353
std,4.179855e+06,415.375438,883.825982,0.208793,27.278442
min,1.309680e+05,164.000000,289.000000,3.400000,904.000000
25%,7.691922e+06,512.750000,999.000000,4.000000,926.000000
50%,1.056080e+07,674.000000,1499.000000,4.200000,950.000000
75%,1.206023e+07,897.500000,1999.000000,4.300000,974.000000
max,1.752457e+07,2924.000000,6999.000000,4.600000,999.000000


In [ ]:
df.isnull().sum()

URL                        0
Product_id                 0
BrandName                  0
Category                   0
Individual_category        0
category_by_Gender         0
Description                0
DiscountPrice (in Rs)    233
OriginalPrice (in Rs)      0
DiscountOffer             53
SizeOption                 0
Ratings                    0
Reviews                    0
dtype: int64

In [ ]:
# Preprocessing
df['DiscountPrice (in Rs)'] = df['DiscountPrice (in Rs)'].fillna(df['DiscountPrice (in Rs)'].mean())
df['Description'] = df['Description'].fillna('')
df['Ratings'] = df['Ratings'].fillna(0)
df['Reviews'] = df['Reviews'].fillna(0)

In [ ]:
df.isnull().sum()

URL                       0
Product_id                0
BrandName                 0
Category                  0
Individual_category       0
category_by_Gender        0
Description               0
DiscountPrice (in Rs)     0
OriginalPrice (in Rs)     0
DiscountOffer            53
SizeOption                0
Ratings                   0
Reviews                   0
dtype: int64

In [ ]:
print("\nDataset after Preprocessing:")
print(df.head())


Dataset after Preprocessing:
                                                 URL  Product_id   BrandName  \
0  https://www.myntra.com/jeans/roadster/roadster...     2296012    Roadster   
1  https://www.myntra.com/track-pants/locomotive/...    13780156  LOCOMOTIVE   
2  https://www.myntra.com/shirts/roadster/roadste...    11895958    Roadster   
3  https://www.myntra.com/shapewear/zivame/zivame...     4335679      Zivame   
4  https://www.myntra.com/tshirts/roadster/roadst...    11690882    Roadster   

                Category Individual_category category_by_Gender  \
0            Bottom Wear               jeans                Men   
1            Bottom Wear         track-pants                Men   
2                Topwear              shirts                Men   
3  Lingerie & Sleep Wear           shapewear              Women   
4                Western             tshirts              Women   

                                         Description  DiscountPrice (in Rs)  \
0  road

In [ ]:
df.columns

Index(['URL', 'Product_id', 'BrandName', 'Category', 'Individual_category',
       'category_by_Gender', 'Description', 'DiscountPrice (in Rs)',
       'OriginalPrice (in Rs)', 'DiscountOffer', 'SizeOption', 'Ratings',
       'Reviews'],
      dtype='object')

#Content-Based Filtering

In [ ]:
# Content-Based Filtering
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Description'])

In [ ]:
# Display the shape of the TF-IDF matrix
print("\nTF-IDF Matrix Shape:")
print(tfidf_matrix.shape)


TF-IDF Matrix Shape:
(499, 527)


#Collaborative Filtering Model

In [ ]:
# Collaborative Filtering
user_ratings = df.pivot_table(index='Product_id', columns='Ratings', values='Reviews').fillna(0)


In [ ]:
# Display the user-item interaction matrix
print("\nUser-Item Interaction Matrix:")
print(user_ratings.head())


User-Item Interaction Matrix:
Ratings     3.4  3.5    3.6  3.7    3.8    3.9  4.0  4.1  4.2  4.3    4.4  \
Product_id                                                                  
130968      0.0  0.0    0.0  0.0    0.0    0.0  0.0  0.0  0.0  0.0  985.0   
596473      0.0  0.0  978.0  0.0    0.0    0.0  0.0  0.0  0.0  0.0    0.0   
782799      0.0  0.0    0.0  0.0  973.0    0.0  0.0  0.0  0.0  0.0    0.0   
851425      0.0  0.0    0.0  0.0    0.0  907.0  0.0  0.0  0.0  0.0    0.0   
976816      0.0  0.0    0.0  0.0    0.0    0.0  0.0  0.0  0.0  0.0    0.0   

Ratings       4.5  4.6  
Product_id              
130968        0.0  0.0  
596473        0.0  0.0  
782799        0.0  0.0  
851425        0.0  0.0  
976816      987.0  0.0  


In [ ]:
# Perform SVD
svd = TruncatedSVD(n_components=10)
latent_matrix = svd.fit_transform(user_ratings)

In [ ]:

# Display the shape of the latent matrix
print("\nLatent Matrix Shape:")
print(latent_matrix.shape)


Latent Matrix Shape:
(499, 10)


#Hybrid Recommendation System

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Min-Max Scaling for combining scores
scaler = MinMaxScaler()


In [ ]:
def recommend_products(input_value, top_n=5):
    # Initialize product as an empty DataFrame
    product = pd.DataFrame()

    # Check if input is URL, Product_id, or Description
    if isinstance(input_value, str) and input_value.startswith('http'):
        product = df[df['URL'] == input_value]
    elif isinstance(input_value, str) and input_value.isnumeric():
        product = df[df['Product_id'] == int(input_value)]
    else:
        product = df[df['Description'].str.contains(input_value, case=False, na=False)]

    if product.empty:
        return "No matching product found."

    product_index = product.index[0]

    print("\nMatched Product:")
    print(product)

    # Content-based recommendation
    cosine_similarities = cosine_similarity(tfidf_matrix[product_index], tfidf_matrix).flatten()

     # Print content-based similarity scores
    # print("\nContent-based Similarities:")
    # print(cosine_similarities)

    # Collaborative filtering recommendation
    product_latent_vector = latent_matrix[product_index].reshape(1, -1)
    collaborative_similarities = cosine_similarity(product_latent_vector, latent_matrix).flatten()

    # Collaborative filtering recommendation
    product_latent_vector = latent_matrix[product_index].reshape(1, -1)
    collaborative_similarities = cosine_similarity(product_latent_vector, latent_matrix).flatten()

    # Normalize scores
    cosine_similarities = scaler.fit_transform(cosine_similarities.reshape(-1, 1)).flatten()
    collaborative_similarities = scaler.fit_transform(collaborative_similarities.reshape(-1, 1)).flatten()

    # Print normalized similarity scores
    # print("\nNormalized Content-based Similarities:")
    # print(cosine_similarities)
    # print("\nNormalized Collaborative Filtering Similarities:")
    # print(collaborative_similarities)


    # Hybrid score
    hybrid_scores = cosine_similarities + collaborative_similarities

    # # Print hybrid scores
    # print("\nHybrid Scores:")
    # print(hybrid_scores)

    # Get top N recommendations
    top_indices = hybrid_scores.argsort()[-top_n:][::-1]
    recommended_products = df.iloc[top_indices]

    return recommended_products[['URL', 'Product_id', 'BrandName', 'Category', 'Description', 'Ratings', 'Reviews']]


In [ ]:

def recommend_products(input_value, top_n=5):
    # Initialize product as an empty DataFrame
    product = pd.DataFrame()

    # Check if input is URL, Product_id, or Description
    if isinstance(input_value, str) and input_value.startswith('http'):
        product = df[df['URL'] == input_value]
    elif isinstance(input_value, str) and input_value.isnumeric():
        product = df[df['Product_id'] == int(input_value)]
    else:
        product = df[df['Description'].str.contains(input_value, case=False, na=False)]

    if product.empty:
        return "No matching product found."

    product_index = product.index[0]

    print("\nMatched Product:")
    print(product)

    # Content-based recommendation
    cosine_similarities = cosine_similarity(tfidf_matrix[product_index], tfidf_matrix).flatten()

    # Collaborative filtering recommendation
    product_latent_vector = latent_matrix[product_index].reshape(1, -1)
    collaborative_similarities = cosine_similarity(product_latent_vector, latent_matrix).flatten()

    # Normalize scores
    cosine_similarities = scaler.fit_transform(cosine_similarities.reshape(-1, 1)).flatten()
    collaborative_similarities = scaler.fit_transform(collaborative_similarities.reshape(-1, 1)).flatten()

    # Hybrid score
    hybrid_scores = cosine_similarities + collaborative_similarities

    # Get top N recommendations
    top_indices = hybrid_scores.argsort()[-top_n:][::-1]
    recommended_products = df.iloc[top_indices]

    return recommended_products[['URL', 'Product_id', 'BrandName', 'Category', 'Description', 'Ratings', 'Reviews']]



#Output

In [ ]:
# Example usage
input_value = 'https://www.myntra.com/jeans/roadster/roadster-men-navy-blue-slim-fit-mid-rise-clean-look-jeans/2296012/buy'
recommendations = recommend_products(input_value)



Matched Product:
                                                 URL  Product_id BrandName  \
0  https://www.myntra.com/jeans/roadster/roadster...     2296012  Roadster   

      Category Individual_category category_by_Gender  \
0  Bottom Wear               jeans                Men   

                                         Description  DiscountPrice (in Rs)  \
0  roadster men navy blue slim fit mid rise clean...                  824.0   

   OriginalPrice (in Rs) DiscountOffer          SizeOption  Ratings  Reviews  
0                   1499       45% OFF  28, 30, 32, 34, 36      3.9      999  


In [ ]:
# Print the recommended products
print("\nRecommended Products:")
print(recommendations)


Recommended Products:
                                                   URL  Product_id  \
0    https://www.myntra.com/jeans/roadster/roadster...     2296012   
350  https://www.myntra.com/jeans/roadster/roadster...    10064543   
494  https://www.myntra.com/jeans/roadster/roadster...    11274476   
161  https://www.myntra.com/jeans/roadster/roadster...     8788237   
366  https://www.myntra.com/jeans/mast--harbour/mas...    13077152   

          BrandName     Category  \
0          Roadster  Bottom Wear   
350        Roadster  Bottom Wear   
494        Roadster  Bottom Wear   
161        Roadster  Bottom Wear   
366  Mast & Harbour  Bottom Wear   

                                           Description  Ratings  Reviews  
0    roadster men navy blue slim fit mid rise clean...      3.9      999  
350  roadster men blue skinny fit mid rise clean lo...      3.8      930  
494  roadster men blue skinny fit mid rise acid was...      3.8      904  
161  roadster men blue skinny fit mid r

In [ ]:
# Example usage
input_value = '2296012'
recommendations = recommend_products(input_value)



Matched Product:
                                                 URL  Product_id BrandName  \
0  https://www.myntra.com/jeans/roadster/roadster...     2296012  Roadster   

      Category Individual_category category_by_Gender  \
0  Bottom Wear               jeans                Men   

                                         Description  DiscountPrice (in Rs)  \
0  roadster men navy blue slim fit mid rise clean...                  824.0   

   OriginalPrice (in Rs) DiscountOffer          SizeOption  Ratings  Reviews  
0                   1499       45% OFF  28, 30, 32, 34, 36      3.9      999  


In [ ]:
# Print the recommended products
print("\nRecommended Products:")
print(recommendations)


Recommended Products:
                                                   URL  Product_id  \
0    https://www.myntra.com/jeans/roadster/roadster...     2296012   
350  https://www.myntra.com/jeans/roadster/roadster...    10064543   
494  https://www.myntra.com/jeans/roadster/roadster...    11274476   
161  https://www.myntra.com/jeans/roadster/roadster...     8788237   
366  https://www.myntra.com/jeans/mast--harbour/mas...    13077152   

          BrandName     Category  \
0          Roadster  Bottom Wear   
350        Roadster  Bottom Wear   
494        Roadster  Bottom Wear   
161        Roadster  Bottom Wear   
366  Mast & Harbour  Bottom Wear   

                                           Description  Ratings  Reviews  
0    roadster men navy blue slim fit mid rise clean...      3.9      999  
350  roadster men blue skinny fit mid rise clean lo...      3.8      930  
494  roadster men blue skinny fit mid rise acid was...      3.8      904  
161  roadster men blue skinny fit mid r

In [ ]:
# Example usage
input_value = 'zivame women grey tummy  thigh shaper zi0pssan03agrey'
recommendations = recommend_products(input_value)



Matched Product:
                                                  URL  Product_id BrandName  \
80  https://www.myntra.com/shapewear/zivame/zivame...     7569310    Zivame   

                 Category Individual_category category_by_Gender  \
80  Lingerie & Sleep Wear           shapewear              Women   

                                          Description  DiscountPrice (in Rs)  \
80  zivame women grey tummy  thigh shaper zi0pssan...                  746.0   

    OriginalPrice (in Rs) DiscountOffer        SizeOption  Ratings  Reviews  
80                    995       25% OFF  S, M, L, XL, XXL      3.9      981  


In [ ]:
# Print the recommended products
print("\nRecommended Products:")
print(recommendations)


Recommended Products:
                                                   URL  Product_id  \
80   https://www.myntra.com/shapewear/zivame/zivame...     7569310   
441  https://www.myntra.com/shapewear/laceandme/lac...    12069880   
100  https://www.myntra.com/tshirts/dressberry/dres...     2522852   
471  https://www.myntra.com/dresses/tokyo-talkies/t...     4924933   
119  https://www.myntra.com/kurtas/jaipur-kurti/jai...    10317827   

         BrandName               Category  \
80          Zivame  Lingerie & Sleep Wear   
441      Laceandme  Lingerie & Sleep Wear   
100     DressBerry                Western   
471  Tokyo Talkies                Western   
119   Jaipur Kurti              Plus Size   

                                           Description  Ratings  Reviews  
80   zivame women grey tummy  thigh shaper zi0pssan...      3.9      981  
441  laceandme women nude coloured solid shaper bri...      4.0      913  
100  dressberry women grey melange printed round ne...      